In [147]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import math
import numpy as np

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("KEY")

In [4]:
def get_results_from_foursquare (query, location, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={location[0]}%2C{location[1]}&sort=DISTANCE&{limit}=5"
    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"]

In [5]:
query = "cafe"
Ironhack_lat=41.3977737
Ironhack_lon=2.1882674
location = [Ironhack_lat, Ironhack_lon]
limit = 10
results = get_results_from_foursquare (query, location, limit)

In [32]:
client = MongoClient("localhost:27017")
db = client.get_database("Ironhack")

In [57]:
companies = db.get_collection("companies")

In [90]:
valencia_companies = list(companies.find({"offices.city":"Valencia"}))
barcelona_companies = list(companies.find({"offices.city":"Barcelona"}))
madrid_companies = list(companies.find({"offices.city":"Madrid"}))
spain_companies = [valencia_companies, barcelona_companies, madrid_companies]

In [116]:
name = []
city = []
office_number = []
address = []
latitude = []
longitude = []
for each_city in spain_companies:
    for company in each_city:
        try:
            for i in range(len(company["offices"])):
                n = 0
                if company["offices"][i]["city"] in ["Valencia", "Barcelona", "Madrid"]:
                    n += 1
                    name.append(company["name"])
                    city.append(company["offices"][i]["city"])
                    office_number.append(n)
                    address.append(company["offices"][i]["address1"])
                    latitude.append(company["offices"][i]["latitude"])
                    longitude.append(company["offices"][i]["longitude"])
        except:
            pass

In [125]:
spain_dict = {"name":name, "city":city, "office_number":office_number, "address":address, "latitude":latitude, "longitude":longitude}

In [126]:
df = pd.DataFrame.from_dict(spain_dict)

In [148]:
df['address'].replace('', np.nan, inplace=True)

In [152]:
df = df.dropna(subset=["address", "latitude", "longitude"], how='all')

In [161]:
def get_coordinates (place):
    try:
        res = requests.get(f"https://geocode.xyz/{place}?json=1").json()
        return [res["longt"],res["latt"]]
    except:
        return [np.nan, np.nan]

In [ ]:
for i in df.index.values.tolist():
    if math.isnan(df.at[i, "latitude"]) or df.at[i, "latitude"]==0:
        address_complete = df.at[i, "address"] + ", " + df.at[i, "city"] + ", Spain"
        coordinates = get_coordinates(address_complete)
        df.at[i, "latitude"]= float(coordinates[1])
        df.at[i, "longitude"]= float(coordinates[0])

In [195]:
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,NaN,39.64073,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,-1.85761,37.2103
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
5,DOMODESK,Madrid,1,C/Arroyo de Fontarron 53,40.403322,-3.651025
6,H2scan,Valencia,1,28486 Westinghouse Place,34.442143,-118.598765
7,Brightcove,Barcelona,1,Avenida Diagonal 640,2.14804,41.3939
8,eyeOS,Barcelona,1,La Rambla 140 4C,41.385486,2.170287
9,Genoom,Barcelona,1,Gran Capitan 2-4,2.06833,41.36719


In [196]:
for i in df.index.values.tolist():
    if df.at[i, "latitude"] < 20:
        latitude = df.at[i, "latitude"]
        longitude = df.at[i, "longitude"]
        df.at[i, "latitude"] = longitude
        df.at[i, "longitude"] = latitude

In [197]:
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,NaN,39.64073,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,37.2103,-1.85761
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
5,DOMODESK,Madrid,1,C/Arroyo de Fontarron 53,40.403322,-3.651025
6,H2scan,Valencia,1,28486 Westinghouse Place,34.442143,-118.598765
7,Brightcove,Barcelona,1,Avenida Diagonal 640,41.3939,2.14804
8,eyeOS,Barcelona,1,La Rambla 140 4C,41.385486,2.170287
9,Genoom,Barcelona,1,Gran Capitan 2-4,41.36719,2.06833


In [198]:
df['latitude'] = df['latitude'].fillna(0)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\3028638243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['latitude'].fillna(0)


In [200]:
df.drop(df[df.latitude < 30].index, inplace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\1358260333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df.latitude < 30].index, inplace=True)


In [209]:
df.drop(df[df.longitude < -10].index, inplace=True)

In [210]:
df.to_csv("Output/spanish_companies.csv")

In [211]:
df = pd.read_csv('Output/spanish_companies.csv')

In [212]:
df = df.set_index('Unnamed: 0')
df.index.name = None